# Scenario 3 Demo: Highway/Rural RTK Evaluation

This notebook demonstrates the GNSS RTK evaluation pipeline for **scenario3** (Highway/Rural).

**Important:** Scenario3 may use **simulated metrics** if real RINEX data is unavailable.
This notebook explicitly checks and displays the `simulated` flag and provenance.

**Steps:**
1. Validate scenario data with `--strict-real`
2. Run RTK evaluation with intent scoring
3. Display comparison results (with simulated flag check)
4. Plot baseline vs optimised horizontal error

In [ ]:
import subprocess
import os
import csv
import json
from pathlib import Path

# Navigate to project root
PROJECT_ROOT = Path(os.getcwd()).parent.parent
GNSS_DIR = PROJECT_ROOT / "CODE" / "gnss"
SCENARIO = "scenario3"

print(f"Project root: {PROJECT_ROOT}")
print(f"GNSS module: {GNSS_DIR}")

## Step 1: Validate Scenario Data (Strict Real Mode)

**Note:** Scenario3 typically fails strict validation - it uses simulated/synthetic data.
The evaluation will proceed with simulated metrics.

In [ ]:
# Run validate_scenario.py --strict-real scenario3
validate_cmd = [
    "python3", str(GNSS_DIR / "validate_scenario.py"),
    "--scenario", SCENARIO,
    "--strict-real"
]

print(f"Running: {' '.join(validate_cmd)}")
print("-" * 60)

result = subprocess.run(validate_cmd, cwd=str(GNSS_DIR), capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")
print("VALIDATION:", "PASSED" if result.returncode == 0 else "FAILED (expected - will use simulated metrics)")

## Step 2: Run RTK Evaluation with Intent Scoring

In [ ]:
# Run rtk_evaluate.py --scenario scenario3 --intent accuracy
eval_cmd = [
    "python3", str(GNSS_DIR / "rtk_evaluate.py"),
    "--scenario", SCENARIO,
    "--intent", "accuracy"
]

print(f"Running: {' '.join(eval_cmd)}")
print("-" * 60)

result = subprocess.run(eval_cmd, cwd=str(GNSS_DIR), capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")

## Step 3: Load and Display comparison.csv (with Simulated Flag)

In [ ]:
# Find the latest evaluation output directory
output_root = PROJECT_ROOT / "OUTPUTs" / SCENARIO / "evaluation"

if output_root.exists():
    eval_dirs = sorted([d for d in output_root.iterdir() if d.is_dir()], reverse=True)
    if eval_dirs:
        latest_eval = eval_dirs[0]
        comparison_csv = latest_eval / "comparison.csv"
        intent_explain = latest_eval / "intent_score_explain.json"
        print(f"Latest evaluation: {latest_eval.name}")
        print(f"Comparison CSV: {comparison_csv}")
    else:
        comparison_csv = None
        intent_explain = None
        print("No evaluation directories found")
else:
    comparison_csv = None
    intent_explain = None
    print(f"Output directory not found: {output_root}")

In [ ]:
# Read and display comparison.csv - CHECK SIMULATED FLAG
comparison_data = {}

if comparison_csv and comparison_csv.exists():
    with open(comparison_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            comparison_data = dict(row)
            break
    
    # Check simulated flag - PROMINENT DISPLAY
    is_simulated = comparison_data.get('simulated', 'False').lower() == 'true'
    
    print("=" * 70)
    print(f"SCENARIO: {comparison_data.get('scenario', 'N/A')}")
    print(f"Timestamp: {comparison_data.get('timestamp', 'N/A')}")
    print("=" * 70)
    
    # PROMINENT SIMULATED FLAG
    if is_simulated:
        print("\n" + "*" * 70)
        print("*  WARNING: SIMULATED=TRUE                                            *")
        print("*  Metrics are from simulated/synthetic data.                        *")
        print("*  Results are for demonstration purposes only.                      *")
        print("*" * 70 + "\n")
    else:
        print(f"\nSimulated: False (Real RINEX data used)")
    
    print(f"Data Source: {comparison_data.get('data_source', 'N/A')}")
    print(f"Intent: {comparison_data.get('intent', 'N/A')}")
    print(f"Intent Score: {comparison_data.get('intent_score', 'N/A')}")
    print("=" * 70)
    print()
    print(f"{'Metric':<35} {'Baseline':>15} {'Optimised':>15}")
    print("-" * 65)
    
    metrics = [
        ("horizontal_error_rms_m", "HPE RMS (m)"),
        ("horizontal_error_p95_m", "HPE P95 (m)"),
        ("vertical_error_rms_m", "VPE RMS (m)"),
        ("fix_rate_pct", "Fix Rate (%)"),
        ("availability_pct", "Availability (%)"),
    ]
    
    for metric_key, metric_name in metrics:
        baseline_val = comparison_data.get(f"baseline_{metric_key}", "")
        optimised_val = comparison_data.get(f"optimised_{metric_key}", "")
        print(f"{metric_name:<35} {baseline_val:>15} {optimised_val:>15}")
else:
    print("comparison.csv not found. Run evaluation first.")

## Step 3b: Display Intent Score Provenance (Simulated Note)

In [ ]:
# Display intent_score_explain.json which contains simulated note
if intent_explain and intent_explain.exists():
    with open(intent_explain, "r", encoding="utf-8") as f:
        explain_data = json.load(f)
    
    print("INTENT SCORE PROVENANCE")
    print("=" * 70)
    print(f"Intent: {explain_data.get('intent', 'N/A')}")
    print(f"Final Score: {explain_data.get('final_score', 'N/A')}")
    print(f"Formula: {explain_data.get('formula', 'N/A')}")
    
    # Check for simulated note - important for scenario3
    simulated_note = explain_data.get('simulated_note')
    if simulated_note:
        print("\n" + "!" * 70)
        print(f"SIMULATED NOTE: {simulated_note}")
        print("!" * 70)
    
    print("\nComputation Steps:")
    for step in explain_data.get('computation_steps', []):
        print(f"  - {step}")
else:
    print("intent_score_explain.json not found.")

## Step 4: Plot Baseline vs Optimised HPE RMS

In [ ]:
import matplotlib.pyplot as plt

if comparison_data:
    baseline_hpe = comparison_data.get("baseline_horizontal_error_rms_m", "")
    optimised_hpe = comparison_data.get("optimised_horizontal_error_rms_m", "")
    is_simulated = comparison_data.get('simulated', 'False').lower() == 'true'
    
    try:
        baseline_val = float(baseline_hpe) if baseline_hpe else 0.0
        optimised_val = float(optimised_hpe) if optimised_hpe else 0.0
    except ValueError:
        baseline_val, optimised_val = 0.0, 0.0
    
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Gray colors for simulated data, colored for real
    colors = ["#95a5a6", "#7f8c8d"] if is_simulated else ["#9b59b6", "#1abc9c"]
    
    bars = ax.bar(["Baseline", "Optimised"], [baseline_val, optimised_val],
                  color=colors, edgecolor="black", linewidth=1.2)
    
    for bar, val in zip(bars, [baseline_val, optimised_val]):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
                f"{val:.4f}" if val > 0 else "N/A",
                ha="center", va="bottom", fontsize=12, fontweight="bold")
    
    ax.set_ylabel("Horizontal Position Error RMS (m)", fontsize=12)
    title = f"Scenario 3: Baseline vs Optimised HPE RMS\nIntent: {comparison_data.get('intent', 'N/A')}"
    if is_simulated:
        title += "\n[SIMULATED DATA]"
    ax.set_title(title, fontsize=14)
    ax.set_ylim(0, max(baseline_val, optimised_val, 0.1) * 1.3)
    ax.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()
else:
    print("No comparison data available for plotting.")

---

**Output Files:**
- `OUTPUTS/scenario3/evaluation/<timestamp>/comparison.csv`
- `OUTPUTS/scenario3/evaluation/<timestamp>/comparison.json`
- `OUTPUTS/scenario3/evaluation/<timestamp>/intent_score_explain.json`

**Note on Simulated Data:**
When `simulated=True`, metrics are computed using deterministic simulation based on
scenario profile parameters, not actual RTKLIB processing of RINEX data.

# Scenario 3 Demo: Highway/Rural RTK Evaluation

This notebook demonstrates the GNSS RTK evaluation pipeline for **scenario3** (Highway/Rural environment).

**Important:** Scenario3 may use **simulated metrics** if real RINEX data is unavailable.
This notebook explicitly checks and displays the `simulated` flag and provenance.

**Steps:**
1. Validate scenario data with `--strict-real`
2. Run RTK evaluation with intent scoring
3. Display comparison results (with simulated flag check)
4. Plot baseline vs optimised horizontal error

In [ ]:
import subprocess
import os
import csv
import json
from pathlib import Path

# Navigate to project root (adjust if running from different location)
PROJECT_ROOT = Path(os.getcwd()).parent.parent
GNSS_DIR = PROJECT_ROOT / "CODE" / "gnss"
SCENARIO = "scenario3"

print(f"Project root: {PROJECT_ROOT}")
print(f"GNSS module: {GNSS_DIR}")

## Step 1: Validate Scenario Data (Strict Real Mode)

**Note:** Scenario3 typically fails strict validation because it uses simulated/synthetic data.
This is expected - the evaluation will proceed with simulated metrics.

In [ ]:
# Run validate_scenario.py --strict-real scenario3
validate_cmd = [
    "python3", str(GNSS_DIR / "validate_scenario.py"),
    "--scenario", SCENARIO,
    "--strict-real"
]

print(f"Running: {' '.join(validate_cmd)}")
print("-" * 60)

result = subprocess.run(
    validate_cmd,
    cwd=str(GNSS_DIR),
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")
validation_passed = result.returncode == 0
print("VALIDATION:", "PASSED" if validation_passed else "FAILED (expected for scenario3 - will use simulated metrics)")

## Step 2: Run RTK Evaluation with Intent Scoring

In [ ]:
# Run rtk_evaluate.py --scenario scenario3 --intent accuracy
eval_cmd = [
    "python3", str(GNSS_DIR / "rtk_evaluate.py"),
    "--scenario", SCENARIO,
    "--intent", "accuracy"
]

print(f"Running: {' '.join(eval_cmd)}")
print("-" * 60)

result = subprocess.run(
    eval_cmd,
    cwd=str(GNSS_DIR),
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")

## Step 3: Load and Display comparison.csv (with Simulated Flag)

In [ ]:
# Find the latest evaluation output directory
output_root = PROJECT_ROOT / "OUTPUTs" / SCENARIO / "evaluation"

if output_root.exists():
    eval_dirs = sorted([d for d in output_root.iterdir() if d.is_dir()], reverse=True)
    if eval_dirs:
        latest_eval = eval_dirs[0]
        comparison_csv = latest_eval / "comparison.csv"
        comparison_json = latest_eval / "comparison.json"
        intent_explain = latest_eval / "intent_score_explain.json"
        print(f"Latest evaluation: {latest_eval.name}")
        print(f"Comparison CSV: {comparison_csv}")
    else:
        print("No evaluation directories found")
        comparison_csv = None
        comparison_json = None
        intent_explain = None
else:
    print(f"Output directory not found: {output_root}")
    print("Run Step 2 first to generate outputs.")
    comparison_csv = None
    comparison_json = None
    intent_explain = None

In [ ]:
# Read and display comparison.csv as a table
# IMPORTANT: Check and display simulated flag prominently

comparison_data = {}

if comparison_csv and comparison_csv.exists():
    with open(comparison_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            comparison_data = dict(row)
            break  # Single row
    
    # Check simulated flag
    is_simulated = comparison_data.get('simulated', 'False').lower() == 'true'
    
    # Display header with prominent simulated warning
    print("=" * 70)
    print(f"SCENARIO: {comparison_data.get('scenario', 'N/A')}")
    print(f"Timestamp: {comparison_data.get('timestamp', 'N/A')}")
    print("=" * 70)
    
    # PROMINENT SIMULATED FLAG DISPLAY
    if is_simulated:
        print("\n" + "*" * 70)
        print("*  WARNING: SIMULATED=TRUE                                            *")
        print("*  Metrics are computed from simulated/synthetic data.               *")
        print("*  Results are for demonstration purposes only.                      *")
        print("*" * 70 + "\n")
    else:
        print(f"\nSimulated: {comparison_data.get('simulated', 'N/A')} (Real RINEX data used)")
    
    print(f"Data Source: {comparison_data.get('data_source', 'N/A')}")
    print(f"Intent: {comparison_data.get('intent', 'N/A')}")
    print(f"Intent Score: {comparison_data.get('intent_score', 'N/A')}")
    print("=" * 70)
    print()
    print(f"{'Metric':<35} {'Baseline':>15} {'Optimised':>15} {'Delta':>15}")
    print("-" * 80)
    
    metrics = [
        ("horizontal_error_rms_m", "HPE RMS (m)"),
        ("horizontal_error_p95_m", "HPE P95 (m)"),
        ("vertical_error_rms_m", "VPE RMS (m)"),
        ("fix_rate_pct", "Fix Rate (%)"),
        ("availability_pct", "Availability (%)"),
        ("ttff_sec", "TTFF (sec)"),
    ]
    
    for metric_key, metric_name in metrics:
        baseline_val = comparison_data.get(f"baseline_{metric_key}", "")
        optimised_val = comparison_data.get(f"optimised_{metric_key}", "")
        delta_key = metric_key.replace("_m", "_delta_m").replace("_pct", "_delta_pct").replace("_sec", "_delta_sec")
        delta_val = comparison_data.get(f"delta_{delta_key}", "")
        print(f"{metric_name:<35} {baseline_val:>15} {optimised_val:>15} {delta_val:>15}")
else:
    print("comparison.csv not found. Run evaluation first.")

## Step 3b: Display Intent Score Provenance (Simulated Note)

In [ ]:
# Display intent_score_explain.json which contains simulated note
if intent_explain and intent_explain.exists():
    with open(intent_explain, "r", encoding="utf-8") as f:
        explain_data = json.load(f)
    
    print("INTENT SCORE PROVENANCE")
    print("=" * 70)
    print(f"Intent: {explain_data.get('intent', 'N/A')}")
    print(f"Final Score: {explain_data.get('final_score', 'N/A')}")
    print(f"Formula: {explain_data.get('formula', 'N/A')}")
    
    # Check for simulated note
    simulated_note = explain_data.get('simulated_note')
    if simulated_note:
        print("\n" + "!" * 70)
        print(f"SIMULATED NOTE: {simulated_note}")
        print("!" * 70)
    
    print("\nComputation Steps:")
    for step in explain_data.get('computation_steps', []):
        print(f"  - {step}")
else:
    print("intent_score_explain.json not found.")

## Step 4: Plot Baseline vs Optimised HPE RMS

In [ ]:
import matplotlib.pyplot as plt

if comparison_data:
    # Extract HPE RMS values
    baseline_hpe = comparison_data.get("baseline_horizontal_error_rms_m", "")
    optimised_hpe = comparison_data.get("optimised_horizontal_error_rms_m", "")
    is_simulated = comparison_data.get('simulated', 'False').lower() == 'true'
    
    # Convert to float (handle empty strings)
    try:
        baseline_val = float(baseline_hpe) if baseline_hpe else 0.0
        optimised_val = float(optimised_hpe) if optimised_hpe else 0.0
    except ValueError:
        baseline_val = 0.0
        optimised_val = 0.0
    
    # Create bar chart
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Use different colors for simulated data
    if is_simulated:
        colors = ["#95a5a6", "#7f8c8d"]  # Gray tones for simulated
    else:
        colors = ["#9b59b6", "#1abc9c"]  # Purple/teal for real
    
    bars = ax.bar(
        ["Baseline", "Optimised"],
        [baseline_val, optimised_val],
        color=colors,
        edgecolor="black",
        linewidth=1.2
    )
    
    # Add value labels on bars
    for bar, val in zip(bars, [baseline_val, optimised_val]):
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.01,
            f"{val:.4f}" if val > 0 else "N/A",
            ha="center", va="bottom", fontsize=12, fontweight="bold"
        )
    
    ax.set_ylabel("Horizontal Position Error RMS (m)", fontsize=12)
    
    # Title includes simulated warning
    title = f"Scenario 3 (Highway/Rural): Baseline vs Optimised HPE RMS\nIntent: {comparison_data.get('intent', 'N/A')}"
    if is_simulated:
        title += "\n[SIMULATED DATA]"
    ax.set_title(title, fontsize=14)
    
    ax.set_ylim(0, max(baseline_val, optimised_val, 0.1) * 1.3)
    ax.grid(axis="y", linestyle="--", alpha=0.7)
    
    plt.tight_layout()
    plt.show()
else:
    print("No comparison data available for plotting.")

---

**Output Files Location:**
- `OUTPUTS/scenario3/evaluation/<eval_timestamp>/comparison.csv`
- `OUTPUTS/scenario3/evaluation/<eval_timestamp>/comparison.json`
- `OUTPUTS/scenario3/evaluation/<eval_timestamp>/intent_score_explain.json`

**Note on Simulated Data:**
When `simulated=True`, metrics are computed using deterministic simulation based on
scenario profile parameters, not actual RTKLIB processing of RINEX data.
This is useful for pipeline testing but should not be used for production evaluation.

# Scenario 3 Demo: Highway/Rural RTK Evaluation

This notebook demonstrates the GNSS RTK evaluation pipeline for **scenario3** (Highway/Rural environment).

**Important:** Scenario3 may use **simulated metrics** if real RINEX data is unavailable.
This notebook explicitly checks and displays the `simulated` flag and provenance.

**Steps:**
1. Validate scenario data with `--strict-real`
2. Run RTK evaluation with intent scoring
3. Display comparison results (with simulated flag check)
4. Plot baseline vs optimised horizontal error

In [ ]:
import subprocess
import os
import csv
import json
from pathlib import Path

# Navigate to project root (adjust if running from different location)
PROJECT_ROOT = Path(os.getcwd()).parent.parent
GNSS_DIR = PROJECT_ROOT / "CODE" / "gnss"
SCENARIO = "scenario3"

print(f"Project root: {PROJECT_ROOT}")
print(f"GNSS module: {GNSS_DIR}")

## Step 1: Validate Scenario Data (Strict Real Mode)

**Note:** Scenario3 typically fails strict validation because it uses simulated/synthetic data.
This is expected - the evaluation will proceed with simulated metrics.

In [ ]:
# Run validate_scenario.py --strict-real scenario3
validate_cmd = [
    "python3", str(GNSS_DIR / "validate_scenario.py"),
    "--scenario", SCENARIO,
    "--strict-real"
]

print(f"Running: {' '.join(validate_cmd)}")
print("-" * 60)

result = subprocess.run(
    validate_cmd,
    cwd=str(GNSS_DIR),
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")
validation_passed = result.returncode == 0
print("VALIDATION:", "PASSED" if validation_passed else "FAILED (expected for scenario3 - will use simulated metrics)")

## Step 2: Run RTK Evaluation with Intent Scoring

In [ ]:
# Run rtk_evaluate.py --scenario scenario3 --intent accuracy
eval_cmd = [
    "python3", str(GNSS_DIR / "rtk_evaluate.py"),
    "--scenario", SCENARIO,
    "--intent", "accuracy"
]

print(f"Running: {' '.join(eval_cmd)}")
print("-" * 60)

result = subprocess.run(
    eval_cmd,
    cwd=str(GNSS_DIR),
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)
print(f"\nReturn code: {result.returncode}")

## Step 3: Load and Display comparison.csv (with Simulated Flag)

In [ ]:
# Find the latest evaluation output directory
output_root = PROJECT_ROOT / "OUTPUTs" / SCENARIO / "evaluation"

if output_root.exists():
    eval_dirs = sorted([d for d in output_root.iterdir() if d.is_dir()], reverse=True)
    if eval_dirs:
        latest_eval = eval_dirs[0]
        comparison_csv = latest_eval / "comparison.csv"
        comparison_json = latest_eval / "comparison.json"
        intent_explain = latest_eval / "intent_score_explain.json"
        print(f"Latest evaluation: {latest_eval.name}")
        print(f"Comparison CSV: {comparison_csv}")
    else:
        print("No evaluation directories found")
        comparison_csv = None
        comparison_json = None
        intent_explain = None
else:
    print(f"Output directory not found: {output_root}")
    print("Run Step 2 first to generate outputs.")
    comparison_csv = None
    comparison_json = None
    intent_explain = None

In [ ]:
# Read and display comparison.csv as a table
# IMPORTANT: Check and display simulated flag prominently

comparison_data = {}

if comparison_csv and comparison_csv.exists():
    with open(comparison_csv, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            comparison_data = dict(row)
            break  # Single row
    
    # Check simulated flag
    is_simulated = comparison_data.get('simulated', 'False').lower() == 'true'
    
    # Display header with prominent simulated warning
    print("=" * 70)
    print(f"SCENARIO: {comparison_data.get('scenario', 'N/A')}")
    print(f"Timestamp: {comparison_data.get('timestamp', 'N/A')}")
    print("=" * 70)
    
    # PROMINENT SIMULATED FLAG DISPLAY
    if is_simulated:
        print("\n" + "*" * 70)
        print("*  WARNING: SIMULATED=TRUE                                            *")
        print("*  Metrics are computed from simulated/synthetic data.               *")
        print("*  Results are for demonstration purposes only.                      *")
        print("*" * 70 + "\n")
    else:
        print(f"\nSimulated: {comparison_data.get('simulated', 'N/A')} (Real RINEX data used)")
    
    print(f"Data Source: {comparison_data.get('data_source', 'N/A')}")
    print(f"Intent: {comparison_data.get('intent', 'N/A')}")
    print(f"Intent Score: {comparison_data.get('intent_score', 'N/A')}")
    print("=" * 70)
    print()
    print(f"{'Metric':<35} {'Baseline':>15} {'Optimised':>15} {'Delta':>15}")
    print("-" * 80)
    
    metrics = [
        ("horizontal_error_rms_m", "HPE RMS (m)"),
        ("horizontal_error_p95_m", "HPE P95 (m)"),
        ("vertical_error_rms_m", "VPE RMS (m)"),
        ("fix_rate_pct", "Fix Rate (%)"),
        ("availability_pct", "Availability (%)"),
        ("ttff_sec", "TTFF (sec)"),
    ]
    
    for metric_key, metric_name in metrics:
        baseline_val = comparison_data.get(f"baseline_{metric_key}", "")
        optimised_val = comparison_data.get(f"optimised_{metric_key}", "")
        delta_key = metric_key.replace("_m", "_delta_m").replace("_pct", "_delta_pct").replace("_sec", "_delta_sec")
        delta_val = comparison_data.get(f"delta_{delta_key}", "")
        print(f"{metric_name:<35} {baseline_val:>15} {optimised_val:>15} {delta_val:>15}")
else:
    print("comparison.csv not found. Run evaluation first.")

## Step 3b: Display Intent Score Provenance (Simulated Note)

In [ ]:
# Display intent_score_explain.json which contains simulated note
if intent_explain and intent_explain.exists():
    with open(intent_explain, "r", encoding="utf-8") as f:
        explain_data = json.load(f)
    
    print("INTENT SCORE PROVENANCE")
    print("=" * 70)
    print(f"Intent: {explain_data.get('intent', 'N/A')}")
    print(f"Final Score: {explain_data.get('final_score', 'N/A')}")
    print(f"Formula: {explain_data.get('formula', 'N/A')}")
    
    # Check for simulated note
    simulated_note = explain_data.get('simulated_note')
    if simulated_note:
        print("\n" + "!" * 70)
        print(f"SIMULATED NOTE: {simulated_note}")
        print("!" * 70)
    
    print("\nComputation Steps:")
    for step in explain_data.get('computation_steps', []):
        print(f"  - {step}")
else:
    print("intent_score_explain.json not found.")

## Step 4: Plot Baseline vs Optimised HPE RMS

In [ ]:
import matplotlib.pyplot as plt

if comparison_data:
    # Extract HPE RMS values
    baseline_hpe = comparison_data.get("baseline_horizontal_error_rms_m", "")
    optimised_hpe = comparison_data.get("optimised_horizontal_error_rms_m", "")
    is_simulated = comparison_data.get('simulated', 'False').lower() == 'true'
    
    # Convert to float (handle empty strings)
    try:
        baseline_val = float(baseline_hpe) if baseline_hpe else 0.0
        optimised_val = float(optimised_hpe) if optimised_hpe else 0.0
    except ValueError:
        baseline_val = 0.0
        optimised_val = 0.0
    
    # Create bar chart
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Use different colors for simulated data
    if is_simulated:
        colors = ["#95a5a6", "#7f8c8d"]  # Gray tones for simulated
    else:
        colors = ["#9b59b6", "#1abc9c"]  # Purple/teal for real
    
    bars = ax.bar(
        ["Baseline", "Optimised"],
        [baseline_val, optimised_val],
        color=colors,
        edgecolor="black",
        linewidth=1.2
    )
    
    # Add value labels on bars
    for bar, val in zip(bars, [baseline_val, optimised_val]):
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.01,
            f"{val:.4f}" if val > 0 else "N/A",
            ha="center", va="bottom", fontsize=12, fontweight="bold"
        )
    
    ax.set_ylabel("Horizontal Position Error RMS (m)", fontsize=12)
    
    # Title includes simulated warning
    title = f"Scenario 3 (Highway/Rural): Baseline vs Optimised HPE RMS\nIntent: {comparison_data.get('intent', 'N/A')}"
    if is_simulated:
        title += "\n[SIMULATED DATA]"
    ax.set_title(title, fontsize=14)
    
    ax.set_ylim(0, max(baseline_val, optimised_val, 0.1) * 1.3)
    ax.grid(axis="y", linestyle="--", alpha=0.7)
    
    plt.tight_layout()
    plt.show()
else:
    print("No comparison data available for plotting.")

---

**Output Files Location:**
- `OUTPUTS/scenario3/evaluation/<eval_timestamp>/comparison.csv`
- `OUTPUTS/scenario3/evaluation/<eval_timestamp>/comparison.json`
- `OUTPUTS/scenario3/evaluation/<eval_timestamp>/intent_score_explain.json`

**Note on Simulated Data:**
When `simulated=True`, metrics are computed using deterministic simulation based on
scenario profile parameters, not actual RTKLIB processing of RINEX data.
This is useful for pipeline testing but should not be used for production evaluation.